In [1]:
import pandas as pd
import datetime

In [2]:
OrderBookPlots2002_df = pd.read_csv('./resources/2002.csv')
OrderBookPlots2002_df = OrderBookPlots2002_df[OrderBookPlots2002_df.DATE.isin(['2002-08-12', '2002-08-13'])]
# OrderBookPlots2002_df

In [3]:
OrderBookPlots2002_df['TRADENO'] = OrderBookPlots2002_df['TRADENO'].astype(pd.Int64Dtype())

In [4]:
OrderBookPlots2002_df.sort_values(by=['NO'], inplace=True)

In [5]:
OrderBookPlots2002_df[(OrderBookPlots2002_df.PRICE != OrderBookPlots2002_df.TRADEPRICE) &
                 pd.notnull(OrderBookPlots2002_df.TRADEPRICE)]

,Unnamed: 0,NO,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME,TRADENO,TRADEPRICE,DATE,OBPLOTNO
1356,1356,1357,RU0009029540,S,103002000,1354,2,0.000,5,29844786,4420.000,2002-08-12,48
1382,1382,1383,RU0008926621,B,103010000,1374,2,2.900,2100,29844787,2.810,2002-08-12,530
1416,1416,1417,RU0008926621,S,103015000,1397,2,2.800,1200,29844788,2.801,2002-08-12,167
1480,1480,1481,RU14MSNG3008,B,103023000,1439,2,0.800,68500,29844794,0.799,2002-08-12,96
1483,1483,1484,RU0008926621,S,103023000,1441,2,2.801,30000,29844795,2.810,2002-08-12,530
...,...,...,...,...,...,...,...,...,...,...,...,...,...
185377,94633,94634,RU0008926621,B,184458000,41551,2,3.600,30000,29878363,2.894,2002-08-13,42428
185379,94635,94636,RU0008926621,B,184458000,41551,2,3.600,100000,29878364,2.894,2002-08-13,42428
185381,94637,94638,RU0008926621,B,184458000,41551,2,3.600,68300,29878365,2.895,2002-08-13,42428
185383,94639,94640,RU0008926621,B,184458000,41551,2,3.600,26900,29878366,2.895,2002-08-13,42428


In [6]:
type(OrderBookPlots2002_df.TIME[0]), type(OrderBookPlots2002_df.TRADENO[0])

(numpy.int64, pandas._libs.missing.NAType)

In [7]:
OrderBookPlots2007_df = pd.read_csv('./resources/LKOH_SBER_GMKN_2007_10.csv')
# OrderBookPlots2007_df = pd.read_csv('./resources/LKOH_2007_10.csv')
OrderBookPlots2007_df = OrderBookPlots2007_df[OrderBookPlots2007_df.DATE.isin(['2007-10-01', '2007-10-02'])]
# OrderBookPlots2007_df

In [8]:
OrderBookPlots2007_df['TRADENO'] = OrderBookPlots2007_df['TRADENO'].astype(pd.Int64Dtype())

In [9]:
OrderBookPlots2007_df.sort_values(by=['NO'], inplace=True)

In [10]:
OrderBookPlots2007_df[(OrderBookPlots2007_df.PRICE != OrderBookPlots2007_df.TRADEPRICE) &
                 pd.notnull(OrderBookPlots2007_df.TRADEPRICE)]

,Unnamed: 0,NO,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME,TRADENO,TRADEPRICE,DATE,OBPLOTNO
2336,15529,15530,LKOH,S,103001000,15442,2,0.00,400,329668177,2066.11,2007-10-01,122
2359,15672,15673,LKOH,B,103002000,15522,2,2069.00,2,329668201,2067.00,2007-10-01,541
2361,15676,15677,LKOH,B,103002000,15524,2,0.00,18,329668203,2067.00,2007-10-01,541
2363,15680,15681,LKOH,S,103002000,15528,2,2060.00,55,329668205,2066.11,2007-10-01,122
2365,15682,15683,LKOH,S,103002000,15581,2,2065.90,100,329668206,2066.11,2007-10-01,122
...,...,...,...,...,...,...,...,...,...,...,...,...,...
521656,1132248,1132249,GMKN,S,174457000,476469,2,6644.00,5,330414209,6644.10,2007-10-02,102950
521658,1132250,1132251,GMKN,S,174457000,476469,2,6644.00,1,330414210,6644.07,2007-10-02,102950
521678,1132345,1132346,SBER03,B,174458000,476517,2,104.94,200,330414238,104.89,2007-10-02,49790
521681,1132350,1132351,LKOH,B,174458000,476519,2,2112.01,5,330414240,2112.00,2007-10-02,63756


In [11]:
OrderBookPlots2002_2000ms_df = OrderBookPlots2002_df[(OrderBookPlots2002_df.TIME >= 103002000) & (OrderBookPlots2002_df.TIME <= 103003000)]
OrderBookPlots2002_2000ms_df

,Unnamed: 0,NO,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME,TRADENO,TRADEPRICE,DATE,OBPLOTNO
1353,1353,1354,RU0008943394,B,103002000,137,0,33.510,5000,<NA>,NaN,2002-08-12,16
1354,1354,1355,RU0009029540,S,103002000,1354,1,0.000,5,<NA>,NaN,2002-08-12,48
1355,1355,1356,RU0009029540,B,103002000,609,2,4420.000,5,29844786,4420.0,2002-08-12,48
1356,1356,1357,RU0009029540,S,103002000,1354,2,0.000,5,29844786,4420.0,2002-08-12,48
1357,1357,1358,RU0008926621,B,103003000,386,0,2.780,4300,<NA>,NaN,2002-08-12,115
1358,1358,1359,RU0008926621,B,103003000,1355,1,2.765,300,<NA>,NaN,2002-08-12,416
92236,1492,1493,RU14MSNG3008,B,103002000,1437,0,0.750,1000,<NA>,NaN,2002-08-13,288
92237,1493,1494,RU0008926621,B,103002000,1493,1,2.770,1500,<NA>,NaN,2002-08-13,631
92238,1494,1495,RU14TATN3006,S,103002000,1494,1,18.990,8800,<NA>,NaN,2002-08-13,16
92239,1495,1496,RU0008943394,B,103003000,1495,1,34.050,200,<NA>,NaN,2002-08-13,277


In [12]:
OrderBookPlots2002_2000ms_df = OrderBookPlots2002_df[(OrderBookPlots2002_df.TIME >= 103002000) & (OrderBookPlots2002_df.TIME <= 103003000)]
def form_datetimemlls(df):
    scnd_size_df = pd.DataFrame(df.groupby('TIME').size())
    scnd_size_df.reset_index(inplace=True)
    scnd_size_df = scnd_size_df.rename(columns = {0:'SCNDSIZE'})
    df = df.merge(scnd_size_df, on='TIME')
    df['SEC_IND'] = df.groupby(['TIME']).cumcount()
    df['MILLISS'] = round(df['SEC_IND'] / df['SCNDSIZE'] * 1000)
    df['MILLISS'] = df['MILLISS'].astype(int)
    df.drop(['SCNDSIZE', 'SEC_IND'], axis=1, inplace=True)
    df['DATETIME'] = df.apply(lambda row: datetime.datetime.strptime(row.DATE, '%Y-%m-%d'), axis=1)
    df['DATETIMEMLLS'] = df.apply(lambda row: datetime.datetime(row.DATETIME.year,
                                                               row.DATETIME.month,
                                                               row.DATETIME.day,
                                                               int(row.TIME // 1e7),
                                                               int((row.TIME // 1e5) % 100),
                                                               int((row.TIME // 1e3) % 100)),
                                  axis=1)
    df['DATETIMEMLLS'] = df.apply(lambda row: row.DATETIMEMLLS +
                                 datetime.timedelta(milliseconds=row.MILLISS),
                                 axis=1)
    df.drop(['DATETIME', 'MILLISS', 'TIME'], axis=1, inplace=True)
    return(df)
OrderBookPlots2002_2000ms_df = form_datetimemlls(OrderBookPlots2002_2000ms_df)
OrderBookPlots2002_2000ms_df

,Unnamed: 0,NO,SECCODE,BUYSELL,ORDERNO,ACTION,PRICE,VOLUME,TRADENO,TRADEPRICE,DATE,OBPLOTNO,DATETIMEMLLS
0,1353,1354,RU0008943394,B,137,0,33.510,5000,<NA>,NaN,2002-08-12,16,2002-08-12 10:30:02.000
1,1354,1355,RU0009029540,S,1354,1,0.000,5,<NA>,NaN,2002-08-12,48,2002-08-12 10:30:02.143
2,1355,1356,RU0009029540,B,609,2,4420.000,5,29844786,4420.0,2002-08-12,48,2002-08-12 10:30:02.286
3,1356,1357,RU0009029540,S,1354,2,0.000,5,29844786,4420.0,2002-08-12,48,2002-08-12 10:30:02.429
4,1492,1493,RU14MSNG3008,B,1437,0,0.750,1000,<NA>,NaN,2002-08-13,288,2002-08-13 10:30:02.571
5,1493,1494,RU0008926621,B,1493,1,2.770,1500,<NA>,NaN,2002-08-13,631,2002-08-13 10:30:02.714
6,1494,1495,RU14TATN3006,S,1494,1,18.990,8800,<NA>,NaN,2002-08-13,16,2002-08-13 10:30:02.857
7,1357,1358,RU0008926621,B,386,0,2.780,4300,<NA>,NaN,2002-08-12,115,2002-08-12 10:30:03.000
8,1358,1359,RU0008926621,B,1355,1,2.765,300,<NA>,NaN,2002-08-12,416,2002-08-12 10:30:03.200
9,1495,1496,RU0008943394,B,1495,1,34.050,200,<NA>,NaN,2002-08-13,277,2002-08-13 10:30:03.400


In [13]:
OrderBookPlots2002_df = form_datetimemlls(OrderBookPlots2002_df)

In [14]:
OrderBookPlots2007_df = form_datetimemlls(OrderBookPlots2007_df)

In [15]:
class OBPlots2OBPlots_Attributes:
    def __init__(self, OBPs_df):
        self.OBPs_df = OBPs_df
        self.secs = pd.unique(self.OBPs_df.SECCODE)
        self.dates = pd.unique(self.OBPs_df.DATE)
        print(self.dates)
        self.obplotnos = pd.unique(self.OBPs_df.OBPLOTNO)
        self.obp_atts = {
            sec:datedic for (sec, datedic) in zip(
                self.secs,
                [{
                    date:obps_attsdic for (date, obps_attsdic) in zip(
                        self.dates, 
                        [{} for i in range(len(self.dates))]
                    )
                } for i in range(len(self.secs))]
            )
        }
        self.iterate_ob_rows()
    
    # Accordin to specs ACTION 0 shows balance of order, not cancelled volune of the order, IT'S WRONG
    # Actually, it shows cancelled volune of the order
    def cur_CBOVOL(self, LAST_CBOVOL, BUYSELL, ACTION, PRICE, VOLUME):
        if (BUYSELL == 'B') & (ACTION == 0):
            return(LAST_CBOVOL + (PRICE * VOLUME))
        else:
            return(LAST_CBOVOL)
        
    def cur_CSOVOL(self, LAST_CSOVOL, BUYSELL, ACTION, PRICE, VOLUME):
        if (BUYSELL == 'S') & (ACTION == 0):
            return(LAST_CSOVOL + (PRICE * VOLUME))
        else:
            return(LAST_CSOVOL)
        
    def cur_BOVOL(self, LAST_BOVOL, BUYSELL, ACTION, PRICE, VOLUME):
        if (BUYSELL == 'B') & (ACTION == 1):
            return(LAST_BOVOL + (PRICE * VOLUME))
        else:
            return(LAST_BOVOL)
        
    def cur_SOVOL(self, LAST_SOVOL, BUYSELL, ACTION, PRICE, VOLUME):
        if (BUYSELL == 'S') & (ACTION == 1):
            return(LAST_SOVOL + (PRICE * VOLUME))
        else:
            return(LAST_SOVOL)
        
    def ob_row2obp_atts(self, SECCODE, DATE, NO, BUYSELL, ORDERNO,
                        ACTION, PRICE, VOLUME, TRADEPRICE, OBPLOTNO,
                        DATETIMEMLLS):
#         print(SECCODE)
#         print(self.obp_atts[SECCODE])
        if OBPLOTNO not in self.obp_atts[SECCODE][DATE]:
            self.obp_atts[SECCODE][DATE][OBPLOTNO] = {}
            
        cur_obps_attsdic_len = len(self.obp_atts[SECCODE][DATE][OBPLOTNO])
        if cur_obps_attsdic_len > 0:
            last_key = list(self.obp_atts[SECCODE][DATE][OBPLOTNO])[-1]
            LAST_CBOVOL = self.obp_atts[SECCODE][DATE][OBPLOTNO][last_key]['CBOVOL']
            LAST_CSOVOL = self.obp_atts[SECCODE][DATE][OBPLOTNO][last_key]['CSOVOL']
            LAST_BOVOL = self.obp_atts[SECCODE][DATE][OBPLOTNO][last_key]['BOVOL']
            LAST_SOVOL = self.obp_atts[SECCODE][DATE][OBPLOTNO][last_key]['SOVOL']
#             LAST_BTVOL = self.obp_atts[SECCODE][DATE][OBPLOTNO][last_key]['BTVOL']
#             LAST_STVOL = self.obp_atts[SECCODE][DATE][OBPLOTNO][last_key]['STVOL']
#             LAST_TBNET = self.obp_atts[SECCODE][DATE][OBPLOTNO][last_key][TBNET]
#             LAST_TSNET = self.obp_atts[SECCODE][DATE][OBPLOTNO][last_key][TSNET]
#             LAST_OBP_BAL = self.obp_atts[SECCODE][DATE][OBPLOTNO][last_key][OBP_BAL]
#             LAST_DAY_BAL = self.obp_atts[SECCODE][DATE][OBPLOTNO][last_key][DAY_BAL]
        else:
            LAST_CBOVOL = 0
            LAST_CSOVOL = 0
            LAST_BOVOL = 0
            LAST_SOVOL = 0
            LAST_BTVOL = 0
            LAST_STVOL = 0
            LAST_TBNET = 0
            LAST_TSNET = 0
            LAST_OBP_BAL = 0
            LAST_DAY_BAL = 0
        self.obp_atts[SECCODE][DATE][OBPLOTNO][NO] = {
            'CBOVOL': self.cur_CBOVOL(LAST_CBOVOL, BUYSELL, ACTION, PRICE, VOLUME), # cancel buy order volume
            'CSOVOL': self.cur_CSOVOL(LAST_CSOVOL, BUYSELL, ACTION, PRICE, VOLUME), # cancel sell order volume
            'BOVOL': self.cur_BOVOL(LAST_BOVOL, BUYSELL, ACTION, PRICE, VOLUME), # buy order volume
            'SOVOL': self.cur_SOVOL(LAST_SOVOL, BUYSELL, ACTION, PRICE, VOLUME)}#, # sell order volume
#             'BTVOL': self.cur_BTVOL(), # buy trade volumne
#             'STVOL': self.cur_STVOL(), # sell trade volume
#             'TBNET': self.cur_TBNET(), # trade volumne - buy volume <= 0
#             'TSNET': self.cur_TSNET(), # trade volume - sell volume >= 0
#             'OBP_BAL': self.cur_OBP_BAL(), # oreder-book-plot balance
#             'DAY_BAL': self.cur_DAY_BAL() } # trading day balance
        
    def iterate_ob_rows(self):
        [self.ob_row2obp_atts(SECCODE, DATE, NO, BUYSELL, ORDERNO, 
                              ACTION, PRICE, VOLUME, TRADEPRICE, OBPLOTNO,
                              DATETIMEMLLS)
        for (SECCODE, DATE, NO, BUYSELL, ORDERNO,
             ACTION, PRICE, VOLUME, TRADEPRICE, OBPLOTNO,
             DATETIMEMLLS)
         in zip(self.OBPs_df.SECCODE,
                self.OBPs_df.DATE,
                self.OBPs_df.NO,
                self.OBPs_df.BUYSELL,
                self.OBPs_df.ORDERNO,
                self.OBPs_df.ACTION,
                self.OBPs_df.PRICE,
                self.OBPs_df.VOLUME,
                self.OBPs_df.TRADEPRICE,
                self.OBPs_df.OBPLOTNO,
                self.OBPs_df.DATETIMEMLLS)]

In [16]:
obp_atts = OBPlots2OBPlots_Attributes(OrderBookPlots2002_2000ms_df)

['2002-08-12' '2002-08-13']


In [17]:
obp_atts.obp_atts.keys()

dict_keys(['RU0008943394', 'RU0009029540', 'RU14MSNG3008', 'RU0008926621', 'RU14TATN3006', 'RU0009029532'])

In [19]:
obp_atts.obp_atts['RU0008943394'].keys()

dict_keys(['2002-08-12', '2002-08-13'])

In [20]:
obp_atts.obp_atts['RU0008943394']['2002-08-12'].keys()

dict_keys([16])

In [25]:
obp_atts.obp_atts['RU0008943394']['2002-08-12'][16].keys()

dict_keys([1354])